In [ ]:
from app.services.hantu_services.auth_service import AuthService


auth_service = AuthService()
# access token 발급
access_token = auth_service.get_access_token()

In [5]:
from app.services.hantu_services.quotations_service import QuotationsService


quotations_service = QuotationsService(access_token)

In [ ]:
# 주식현재가 시세
inquire_price_res = quotations_service.inquire_price("005930")

In [ ]:
print(
    inquire_price_res["stck_prpr"],  # 주식 현재가
    inquire_price_res["prdy_vrss"],  # 전일대비
    inquire_price_res["prdy_ctrt"],  # 전일대비율
    inquire_price_res[
        "prdy_vrss_sign"
    ],  # 전일 대비 부호 1: 상한, 2: 상승, 3: 보합, 4: 하한, 5: 하락
    inquire_price_res["stck_oprc"],  # 주식 시가
    inquire_price_res["stck_hgpr"],  # 주식 최고가
    inquire_price_res["stck_lwpr"],  # 주식 최저가
    inquire_price_res["stck_mxpr"],  # 주식 상한가
    inquire_price_res["stck_llam"],  # 주식 하한가
    inquire_price_res["stck_sdpr"],  # 주식 기준가
    inquire_price_res[
        "mrkt_warn_cls_code"
    ],  # 시장경고코드 00: 없음, 01: 투자주의, 02: 투자경고, 03: 투자위험
)

In [8]:
from app.services.hantu_services.trading_service import TradingService


trading_service = TradingService(access_token)

In [9]:
# 주식주문(현금)
order_cash_res = trading_service.order_cash("005930", "1", "40000")

In [ ]:
# 주식일별주문체결조회
inquire_daily_ccld_res = trading_service.inquire_daily_ccld()
print(inquire_daily_ccld_res)
# inquire_daily_ccld_res[0]["odno"]  # 주문번호

In [ ]:
def filter_ticker(obj):
    return True if obj["pdno"] == "005930" else False


filtered_list = list(filter(filter_ticker, inquire_daily_ccld_res))

if len(filtered_list) > 0:
    # 주식주문(정정취소)
    order_rvsenclcl_res = trading_service.order_rvsecncl(filtered_list[0]["odno"])
    print(order_rvsenclcl_res["msg_cd"], order_rvsenclcl_res["msg1"])
else:
    print("주문번호가 존재하지 않습니다.")

In [2]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

engine = create_engine("sqlite:///./app/connection/test.db", echo=False)
SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)
session = SessionLocal()

In [ ]:
import pandas as pd
from sqlalchemy import func
from app.entities.kor_ticker_entity import KorTickerEntity


max_subquery = session.query(func.max(KorTickerEntity.baseDt))
query_set = session.query(KorTickerEntity).filter(
    KorTickerEntity.baseDt == max_subquery.scalar_subquery(),
    KorTickerEntity.itemCtg == "보통주",
)
df = pd.read_sql(query_set.statement, query_set.session.bind)

In [8]:
# 시총 작은 순위로 20개 정렬
df_rank = df["mrktTotAmt"].rank(axis=0, ascending=True)
result = df[df_rank < 20].sort_values(by=["mrktTotAmt"], ascending=True)

In [ ]:
result